# Spreedly.ai Demo

This notebook is "inspired" by 🦜⛓️'s OpenAPI [agents](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/openapi.html) and [chains](https://github.com/hwchase17/langchain/blob/master/docs/modules/chains/examples/openapi.ipynb) examples.

## Requirements

- Ensure you have selected the Jupyter-provided `venv`
- Remember to run `python3 -m venv venv`
- Create a `.env` with the following format:
```
SPREEDLY_OPEN_API_SPEC_YAML_LOC=path/to/spec.yaml
SPREEDLY_ACCESS_TOKEN=<base64 encoded user:pass>
OPENAI_API_KEY=<your OPEN AI API KEY>
```

## Load our API

In [1]:
import os, yaml

yaml_location =  os.getenv('SPREEDLY_OPEN_API_SPEC_YAML_LOC')
with open(yaml_location) as f:
    spreedly_openai_api_spec = yaml.load(f, Loader=yaml.Loader)

In [2]:
from langchain.requests import RequestsWrapper

access_token = os.getenv('SPREEDLY_ACCESS_TOKEN')
headers = {
        'Authorization': f'Basic {access_token}'
    }
requests_wrapper = RequestsWrapper(headers=headers)




Kick the tires, can we instrospect the API?

In [3]:
endpoints = [
    (route, operation)
    for route, operations in spreedly_openai_api_spec["paths"].items()
    for operation in operations
    if operation in ["get", "post"]
]
display(f"There are {len(endpoints)} endpoints available")

display(f"""Servers are {spreedly_openai_api_spec["servers"][0]["url"]}""")

'There are 5 endpoints available'

'Servers are https://core.spreedly.com/v1'

Prepare our objects

In [4]:
from langchain.chat_models import ChatOpenAI
import planner

# relies on OPENAI_API_KEY from .env file.
# llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)

In [5]:
from langchain.tools import OpenAPISpec, APIOperation
from langchain.chains import OpenAPIEndpointChain
from langchain.requests import Requests
from langchain.llms import OpenAI
llm = OpenAI() # Load a Language Model



# Attempt to chunk the original large YAML...no cigar.

In [6]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
import json

llm = OpenAI(temperature=0)

text_splitter = CharacterTextSplitter(separator = "\n\n",
    chunk_size = 10000,
    chunk_overlap  = 0,
    length_function = len,)
spec = f"{spreedly_openai_api_spec}"

texts = text_splitter.split_text(spec)

from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts]

Sample data

In [7]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm, chain_type="map_reduce")
# chain.run(docs)

endpoint_docs_by_name = {name: docs for name, docs in spreedly_openai_api_spec['paths'].items()}
ep = "/gateways_options.json"
if not endpoint_docs_by_name.get(ep):
    raise ValueError(f"{ep} endpoint does not exist.")

display(f"""{spreedly_openai_api_spec["servers"][0]["url"]}/gateways.json """)

'https://core.spreedly.com/v1/gateways.json '

Create Agent and run query

In [10]:
spreedly_agent = planner.create_openapi_agent(spreedly_openai_api_spec, requests_wrapper, llm)
user_query = "Get me a list of supported gateways"

spreedly_agent.run(user_query)

["/gateways_options.json {'get': {'tags': ['Gateways'], 'operationId': 'gateways_options.list', 'summary': 'List supported gateways', 'description': 'Retrieve a list of all gateways, and their properties, supported by Spreedly. This call can be used to dynamically present a gateway credential form to users or to do any sort of dynamic rendering.\\n\\nNote that this request is publicly available and _does not_ require authorization.\\n', 'security': [], 'responses': {'200': {'description': 'List of supported gateways', 'content': {'application/json': {'schema': {'type': 'object', 'properties': {'gateways': {'type': 'array', 'items': {'type': 'object', 'properties': {'gateway_type': {'description': 'The type (short name) of the gateway. Use this value when provisioning a gateway.', 'type': 'string'}, 'name': {'description': 'The human readable name of the gateway', 'type': 'string'}, 'auth_modes': {'description': 'A list of authorization modes for the gateway. An auth mode is basically a

'A list of gateways and their properties, including ids and names for each gateway.'